In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

# 1. 自定义Dataset类，处理不等长序列
class MolecularDataset(Dataset):
    def __init__(self, input_sequences):
        self.input_sequences = input_sequences
        self.max_len = max(len(seq) for seq in input_sequences)  # 找到最长序列长度

    def __len__(self):
        return len(self.input_sequences)

    def __getitem__(self, idx):
        sequence = self.input_sequences[idx]
        # 填充序列至最大长度
        padded_sequence = sequence + [[0] * len(sequence[0])] * (self.max_len - len(sequence))
        return torch.tensor(padded_sequence, dtype=torch.float32), len(sequence)

# 2. 定义一个Transformer模型来提取特征
class MolecularEmbeddingModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
        super(MolecularEmbeddingModel, self).__init__()
        self.embedding = nn.Linear(input_dim, embedding_dim)  # 特征嵌入层
        self.transformer = nn.Transformer(
            d_model=embedding_dim,  # 输入维度
            nhead=num_heads,        # 注意力头的数量
            num_encoder_layers=num_layers  # Transformer层数
        )
        self.decoder = nn.Linear(embedding_dim, input_dim)  # 解码器，用于重建输入（自监督）

    def forward(self, x):
        # 将输入嵌入到更高维度
        embedded = self.embedding(x)
        # Transformer的输入需要是(batch_size, seq_len, embedding_dim)，
        # 但是transformer模型期望的输入是(seq_len, batch_size, embedding_dim)
        embedded = embedded.permute(1, 0, 2)  # 转换成 (seq_len, batch_size, embedding_dim)
        
        # 使用Transformer模型
        transformer_output = self.transformer(embedded)
        
        # 取Transformer输出的最后一层作为分子特征（或者你可以选择其他层的输出）
        return transformer_output[-1, :, :]  # 取最后一时刻的特征

# 3. 初始化数据和模型
# 示例数据，假设每个分子有3个特征 (E1, E2, E3) + 持续时间特征
input_sequences = [
    [[1.0, 2.0, 3.0, 10], [2.0, 3.0, 4.0, 20]],  # 长度为2
    [[2.0, 3.0, 4.0, 15], [3.0, 4.0, 5.0, 25], [4.0, 5.0, 6.0, 30]],  # 长度为3
    [[1.5, 2.5, 3.5, 12]]  # 长度为1
]

# 初始化Dataset和DataLoader
dataset = MolecularDataset(input_sequences)
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

# 初始化模型
input_dim = 4  # 假设每个分子有4个特征（3个化学成分 + 1个持续时间）
embedding_dim = 128  # 特征嵌入维度
num_heads = 4  # 自注意力头数量
num_layers = 2  # Transformer层数
model = MolecularEmbeddingModel(input_dim, embedding_dim, num_heads, num_layers)

# 4. 模型前向传播并提取特征
model.eval()  # 设置为评估模式
with torch.no_grad():
    for batch_idx, (seq, seq_len) in enumerate(dataloader):
        output = model(seq)
        print(f"Batch {batch_idx+1} Output: {output.shape}")
        # 输出每个序列的分子特征（每个分子的嵌入表示）
        print(output)
